#**스마트폰 센서 데이터 기반 모션 분류**
# 단계2 : 기본 모델링


## 0.미션

* 데이터 전처리
    * 가변수화, 데이터 분할, NaN 확인 및 조치, 스케일링 등 필요한 전처리 수행
* 다양한 알고리즘으로 분류 모델 생성
    * 최소 4개 이상의 알고리즘을 적용하여 모델링 수행
    * 성능 비교
        * 각 모델의 성능을 관리하는 별도의 엑셀파일을 만들어 봅시다.
        * 성능 가이드 : Accuracy 0.900 ~

## 1.환경설정

* 세부 요구사항
    - 경로 설정 : 로컬 수행(Ananconda)
        * 제공된 압축파일을 다운받아 압축을 풀고
        * anaconda의 root directory(보통 C:/Users/< ID > 에 project3_1 폴더를 만들고, 복사해 넣습니다.
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
        * 필요하다고 판단되는 라이브러리를 추가하세요.


### (1) 라이브러리 로딩

In [138]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

import joblib

# 필요한 라이브러리, 함수 로딩 ------------------
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import *
# 하이퍼 파라미터 튜닝은 _val로 진행
# 모델 성능 비교는 data_test로 진행
# 변수를 삭제?  > 타당한 근거가 있어야 함  ex) Tree계열 알고리즘은 변수가 많아도 안정적인 성능
#                                               LGBM ,로지스틱 회귀 , KNN은 변수가 많아지거나 이상한 변수가 추가되면 성능하락
#                                               옵션(주성분 분석)


* 제공 함수 생성
    * 변수 중요도를 시각화할 수 있는 함수를 제공합니다.
    * 입력 :
        * importance : 트리모델의 변수 중요도(예: model.feature_importances_)
        * names : 변수 이름 목록(예 : x_train.columns
        * result_only  : 변수 중요도 순으로 데이터프레임만 return할지, 그래프도 포함할지 결정. False이면 결과 데이터프레임 + 그래프
        * topn : 중요도 상위 n개만 표시. all 이면 전체.
    * 출력 :
        * 중요도 그래프 : 중요도 내림차순으로 정렬
        * 중요도 데이터프레임 : 중요도 내림차순으로 정렬

In [140]:
# 변수의 특성 중요도 계산하기
def plot_feature_importance(importance, names, result_only = False, topn = 'all'):
    feature_importance = np.array(importance)
    feature_name = np.array(names)
    
    data={'feature_name':feature_name,'feature_importance':feature_importance}
    fi_temp = pd.DataFrame(data)

    #변수의 특성 중요도 순으로 정렬하기
    fi_temp.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_temp.reset_index(drop=True, inplace = True)

    if topn == 'all' :
        fi_df = fi_temp.copy()
    else :
        fi_df = fi_temp.iloc[:topn]

    #변수의 특성 중요도 그래프로 그리기
    if result_only == False :
        plt.figure(figsize=(10,20))
        sns.barplot(x='feature_importance', y='feature_name', data = fi_df)

        plt.xlabel('importance')
        plt.ylabel('feature name')
        plt.grid()

    return fi_df

### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용
    * data01_test.csv : 테스트용

* 세부 요구사항
    * 칼럼 삭제 : data01_train.csv와 data01_test.csv 에서 'subject' 칼럼은 불필요하므로 삭제합니다.

#### 1) 데이터로딩

In [142]:
data_train = pd.read_csv('data01_train.csv')

In [143]:
data_test = pd.read_csv('data01_test.csv')

In [144]:
drop_cols = 'subject'
data_test.drop(columns = drop_cols ,inplace =True)
data_train.drop(columns = drop_cols, inplace = True)

#### 2) 기본 정보 조회

In [149]:
print(data_train.shape)
print(data_test.shape)

(5881, 562)
(1471, 562)


In [151]:
data_test.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1471 entries, 0 to 1470
Columns: 562 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), object(1)
memory usage: 6.3+ MB


In [153]:
# Acrivity 범주의 종류 확인
categories_train = data_train['Activity'].unique()
categories_test = data_test['Activity'].unique()
print("train 데이터 범주의 종류: \n", categories_train)
print("test 데이터 범주의 종류: \n", categories_train)

train 데이터 범주의 종류: 
 ['STANDING' 'LAYING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPSTAIRS'
 'SITTING']
test 데이터 범주의 종류: 
 ['STANDING' 'LAYING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPSTAIRS'
 'SITTING']


In [155]:
# 범주별 빈도수 확인
activity_count_train = data_train['Activity'].value_counts()
activity_count_test = data_test['Activity'].value_counts()
print("train 데이터 범주별 빈도수: \n", activity_count_train)
print("test 데이터 범주별 빈도수: \n", activity_count_test)

train 데이터 범주별 빈도수: 
 Activity
LAYING                1115
STANDING              1087
SITTING               1032
WALKING                998
WALKING_UPSTAIRS       858
WALKING_DOWNSTAIRS     791
Name: count, dtype: int64
test 데이터 범주별 빈도수: 
 Activity
LAYING                292
STANDING              287
SITTING               254
WALKING               228
WALKING_UPSTAIRS      215
WALKING_DOWNSTAIRS    195
Name: count, dtype: int64


In [157]:
# 범주별 비율 계산
activity_ratio_train = data_train['Activity'].value_counts(normalize=True)
activity_ratio_test = data_test['Activity'].value_counts(normalize=True)
print("train 데이터 범주별 비율(%) \n", activity_ratio_train)
print("test 데이터 범주별 비율(%) \n", activity_ratio_test)

train 데이터 범주별 비율(%) 
 Activity
LAYING                0.189594
STANDING              0.184833
SITTING               0.175480
WALKING               0.169699
WALKING_UPSTAIRS      0.145894
WALKING_DOWNSTAIRS    0.134501
Name: proportion, dtype: float64
test 데이터 범주별 비율(%) 
 Activity
LAYING                0.198504
STANDING              0.195105
SITTING               0.172672
WALKING               0.154997
WALKING_UPSTAIRS      0.146159
WALKING_DOWNSTAIRS    0.132563
Name: proportion, dtype: float64


## **2. 데이터 전처리**

* 가변수화, 데이터 분할, NaN 확인 및 조치, 스케일링 등 필요한 전처리를 수행한다.


### (1) 데이터 분할1 : x, y

* 세부 요구사항
    - x, y로 분할합니다.

In [162]:
target = 'Activity'
x = data_train.drop(columns= target)
y = data_train.loc[:,target]


### (2) 데이터분할2 : train, validation

* 세부 요구사항
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [166]:
from sklearn.model_selection import train_test_split

x_train , x_val , y_train , y_val = train_test_split(x,y,random_state=42,test_size=0.2)

### (3) 스케일링


* 세부 요구사항
    - 스케일링을 필요로 하는 알고리즘 사용을 위해서 코드 수행
    - min-max 방식 혹은 standard 방식 중 한가지 사용.

In [170]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train) # 학습용 데이터는 fit 학습을 수행하고 변환하지만
x_val= scaler.transform(x_val)# 검증용 데이터는 fit 을 수행해선 안됀다 > 평가용 데이터 역시 fit을 해선 안됀다 


### (4) (옵션)  Activity 라벨링 


* 세부 요구사항
    - 데이터 라벨링을 필요로 하는 알고리즘(lightGBM, 또는 향후 딥러닝을) 알고리즘을 위해서 코드 수행
    - Activity 의 6가지 범주를 0~5사이 범주로 대체 

In [174]:
# 사전작업  y label은 연속형 변수로 라벨링해야합니다
# 방법1. # 레이블 인코딩
# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# y_train_encoded = label_encoder.fit_transform(y_train)
# y_val_encoded = label_encoder.fit_transform(y_val)

#방법 2  대체 
activity_mapping = {
    'LAYING': 0,
    'STANDING': 1,
    'SITTING': 2,
    'WALKING': 3,
    'WALKING_UPSTAIRS': 4,
    'WALKING_DOWNSTAIRS': 5
}
y_train = y_train.map(activity_mapping)
y_val = y_val.map(activity_mapping)

## **3. 기본 모델링**



* 세부 요구사항
    - 최소 5개 이상의 알고리즘을 적용하여 모델링을 수행한다.
    - 각 알고리즘 별로 다음 중 몇가지를 시도하며 성능을 비교한다.

### (1) 모델1 : KNN 근접이웃

In [40]:
# KNN 근접 이웃 방식 
# 찾아야하는 param : n_neighbors(초기 n값 ) =? , weights(가중치 방식) = ?  , metric(거리계산 방식) =?  
param_grid = {
    'n_neighbors': range(1,10,2),  #홀수를 자주 사용하는 이유는 다수결 투표에서 동률을 피하기 위해서
    'weights': ['uniform', 'distance'],  # 가중치 방식
    'metric': ['euclidean', 'manhattan', 'minkowski']  # 거리 계산 방식
}
# 모델 생성 
knn = KNeighborsClassifier()

# GridSearchCV를 사용한 하이퍼파라미터 튜닝
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy') # n_jobs = -1 : 모든 CPU가동 병렬처리> 속도향상 하지만 유료 서비스에선 비용발생 
grid_search.fit(x_train, y_train) # x는 반드시 스케일링 

# 최적의 하이퍼파라미터 출력
print(f'최적의 파라미터: {grid_search.best_params_}')


# 최적의 하이퍼파라미터로 학습된 모델을 사용하여 예측
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(x_val)

# 정확도 평가
print(confusion_matrix(y_val, y_pred))
print('Classification Report  \n' , classification_report(y_val, y_pred)) #

최적의 파라미터: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
[[231   0   0   0   0   0]
 [  0 220   6   0   0   0]
 [  1  14 185   0   0   0]
 [  0   0   0 198   0   0]
 [  0   0   0   0 176   1]
 [  0   0   0   0   0 145]]
Classification Report  
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       231
           1       0.94      0.97      0.96       226
           2       0.97      0.93      0.95       200
           3       1.00      1.00      1.00       198
           4       1.00      0.99      1.00       177
           5       0.99      1.00      1.00       145

    accuracy                           0.98      1177
   macro avg       0.98      0.98      0.98      1177
weighted avg       0.98      0.98      0.98      1177



In [41]:
# 최적의 파라미터는 {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}

In [42]:
# 시간 제약 {'metric': 'euclidean', 'n_neighbors': 1}

### (2) 모델2: SVM 서포트 벡터 머신

In [44]:
# 서포트 벡터 머신
param_grid = {
    #'C': [0.1, 1, 10, 100],  # 규제 강도, 작은 값은 과적합을 방지하고 큰 값은 과적합을 허용
    #'kernel': ['linear', 'rbf', 'poly'],  # 커널 함수 (linear, RBF, polynomial)
    #'gamma': ['scale', 'auto', 0.1, 1, 10],  # RBF 커널에서 중요한 파라미터, 다른 커널에서는 무시됨
    'C' : [10],
    'kernel' : ['rbf'],
    'gamma': [0.1]  # RBF 커널에서 중요한 파라미터, 다른 커널에서는 무시됨
}
# 모델 생성 
svm = SVC()

# GridSearchCV를 사용한 하이퍼파라미터 튜닝
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy', n_jobs=1)
grid_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터 출력
print(f'최적의 파라미터: {grid_search.best_params_}')

# 최적의 하이퍼파라미터로 학습된 모델을 사용하여 예측
best_svm  = grid_search.best_estimator_
y_pred = best_knn.predict(x_val)

# 정확도 평가
print(confusion_matrix(y_val, y_pred))
print('Classification Report  \n' , classification_report(y_val, y_pred ))

최적의 파라미터: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
[[231   0   0   0   0   0]
 [  0 220   6   0   0   0]
 [  1  14 185   0   0   0]
 [  0   0   0 198   0   0]
 [  0   0   0   0 176   1]
 [  0   0   0   0   0 145]]
Classification Report  
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       231
           1       0.94      0.97      0.96       226
           2       0.97      0.93      0.95       200
           3       1.00      1.00      1.00       198
           4       1.00      0.99      1.00       177
           5       0.99      1.00      1.00       145

    accuracy                           0.98      1177
   macro avg       0.98      0.98      0.98      1177
weighted avg       0.98      0.98      0.98      1177



In [45]:
# 최적의 파라미터: {'C': 10 , 'kernel': 'rbf' , gammma : 0.1 }

### (3) 모델3:  로지스틱 회귀 (분류모델)

In [47]:
# 파라미터 
param_grid = {
    #'C': [0.01, 0.1, 1, 10, 100], # 규제강도
    #'penalty': ['l1', 'l2', 'elasticnet', 'none'], # 규제 유형 라쏘 ,릿지 , 엘라스틱 , none 
    # 'solver': ['liblinear', 'saga', 'lbfgs', 'newton-cg'], # 최적화 알고리즘
    # 'max_iter': [100, 200, 500, 1000]  # 반복 횟수도 조정 가능
    'C' : [10],
    'penalty' :['l2'],
    'solver':['newton-cg']
    
}

model = LogisticRegression(max_iter = 100)

# GridSearchCV를 사용한 하이퍼파라미터 튜닝
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=1)
grid_search.fit(x_train, y_train) # scaling 된 데이터로 학습해도 되지만 성능 비교를 위해 scaling되지 않은 데이터 부터 학습 

# 최적의 하이퍼파라미터 출력
print(f'최적의 파라미터: {grid_search.best_params_}')

# 최적의 하이퍼파라미터로 학습된 모델을 사용하여 예측
best_log = grid_search.best_estimator_
y_pred = best_knn.predict(x_val)

# 정확도 평가
print(confusion_matrix(y_val, y_pred))
print('Classification Report  \n' , classification_report(y_val, y_pred ))


최적의 파라미터: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
[[231   0   0   0   0   0]
 [  0 220   6   0   0   0]
 [  1  14 185   0   0   0]
 [  0   0   0 198   0   0]
 [  0   0   0   0 176   1]
 [  0   0   0   0   0 145]]
Classification Report  
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       231
           1       0.94      0.97      0.96       226
           2       0.97      0.93      0.95       200
           3       1.00      1.00      1.00       198
           4       1.00      0.99      1.00       177
           5       0.99      1.00      1.00       145

    accuracy                           0.98      1177
   macro avg       0.98      0.98      0.98      1177
weighted avg       0.98      0.98      0.98      1177



In [48]:
 # 최적의 파라미터 : {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}

### (4) 모델4: XGBoost

In [50]:
y_train.value_counts()

Activity
0    884
1    861
2    832
3    800
4    681
5    646
Name: count, dtype: int64

In [51]:
# 그라디언트 부스팅 XGB

xgb = XGBClassifier()

# 하이퍼파라미터 그리드 설정
param_grid = {
    #'n_estimators': [100, 200, 300],  # 트리 개수
    #'learning_rate': [0.01, 0.1, 0.3],  # 학습률
    #'max_depth': [3, 5, 7],  # 트리의 깊이
    #'subsample': [0.7, 0.8, 1.0],  # 데이터를 샘플링할 비율
    #'colsample_bytree': [0.7, 0.8, 1.0]  # 트리에서 사용할 특성 비율
    'n_estimators':[200],
    'max_depth':[3],
    'learning_rate': [0.3],
    'subsample': [0.8],
    'colsample_bytree': [1.0] 
    
}

# GridSearchCV를 사용한 하이퍼파라미터 튜닝
grid_search = GridSearchCV(xgb, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train) # 범주형(y train) 데이터를 0~5로 라벨링 

# 최적의 하이퍼파라미터 출력
print(f'Best Parameters: {grid_search.best_params_}')
# 최적의 하이퍼파라미터로 학습된 모델을 사용하여 예측
best_xgb = grid_search.best_estimator_
y_pred = best_xgb.predict(x_val)

# 정확도 평가
print(confusion_matrix(y_val, y_pred))
print('Classification Report  \n' , classification_report(y_val, y_pred ))

Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}
[[231   0   0   0   0   0]
 [  0 221   5   0   0   0]
 [  0   1 199   0   0   0]
 [  0   0   0 197   1   0]
 [  0   0   0   0 177   0]
 [  0   0   0   0   0 145]]
Classification Report  
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       231
           1       1.00      0.98      0.99       226
           2       0.98      0.99      0.99       200
           3       1.00      0.99      1.00       198
           4       0.99      1.00      1.00       177
           5       1.00      1.00      1.00       145

    accuracy                           0.99      1177
   macro avg       0.99      0.99      0.99      1177
weighted avg       0.99      0.99      0.99      1177



In [52]:
# Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}

### (5) 모델5 : lightGBM (이찬울님 의견)

In [54]:
# 사전 학습용 데이터 가변수화
#x_train.info() # x 는 전부 int() 연속형 데이터들이므로 가변수화를 하지 않아도 됩니다 
# 

AttributeError: 'numpy.ndarray' object has no attribute 'info'

In [122]:
best_lgbm = LGBMClassifier(n_estimators=260 ,learning_rate = 0.422,verbose =-1)# 최적의 파리미터는 이미 생성했습니다 
best_lgbm.fit(x_train,y_train)

y_pred = best_lgbm.predict(x_val)

# 정확도 평가(검증용)
print('============검증 결과==================')
print(confusion_matrix(y_val, y_pred))
print('Classification Report  \n' , classification_report(y_val, y_pred ))

y_pred = model_lgbm.predict(x_test)

# 정확도 평가(평가용)
print('============평가 결과==================')
print(confusion_matrix(y_test, y_pred))
print('Classification Report  \n' , classification_report(y_test, y_pred ))

============검증 결과==================
[[231   0   0   0   0   0]
 [  0 222   4   0   0   0]
 [  0   1 199   0   0   0]
 [  0   0   0 197   1   0]
 [  0   0   0   0 177   0]
 [  0   0   0   0   0 145]]
Classification Report  
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       231
           1       1.00      0.98      0.99       226
           2       0.98      0.99      0.99       200
           3       1.00      0.99      1.00       198
           4       0.99      1.00      1.00       177
           5       1.00      1.00      1.00       145

    accuracy                           0.99      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       0.99      0.99      0.99      1177

============평가 결과==================
[[291   0   0   0   0   1]
 [  0 281   6   0   0   0]
 [  0   5 249   0   0   0]
 [  0   0   0 227   1   0]
 [  0   0   0   0 215   0]
 [  0   0   0   1   0 194]]
Classification Report  
          

In [ ]:
# LightGBM을 하려면 컬럼명도 전부 바꿔줘야 해서 일단 skip
# lgbm = LGBMClassifier()

# # 하이퍼파라미터 그리드 설정
# param_grid = {
#     #'num_leaves': [31, 50, 100],  # 트리의 리프 개수
#     #'learning_rate': [0.01, 0.05, 0.1],  # 학습률
#     'n_estimators': [100, 200, 300],  # 트리 개수
#     #'max_depth': [-1, 5, 10],  # 트리의 최대 깊이 (-1은 제한 없음)
#     #'subsample': [0.7, 0.8, 1.0],  # 데이터를 샘플링할 비율
#     #'colsample_bytree': [0.7, 0.8, 1.0]  # 트리에서 사용할 특성 비율
# }

# # GridSearchCV를 사용한 하이퍼파라미터 튜닝
# grid_search = GridSearchCV(lgbm, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
# grid_search.fit(x_train, y_train_encoded)

# # 최적의 하이퍼파라미터 출력
# print(f'Best Parameters: {grid_search.best_params_}')

# # 최적의 하이퍼파라미터로 학습된 모델을 사용하여 예측
# best_lgbm = grid_search.best_estimator_
# y_pred = best_lgbm.predict(x_val)

# # 정확도 평가
# print(confusion_matrix(y_val_encoded, y_pred))
# print('Classification Report  \n' , classification_report(y_val_encoded, y_pred ))

## 4.성능비교

* 세부 요구사항
    - 각 모델에 대해서 test 데이터로 성능 측정후 비교
    

In [55]:
data_test.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.284379,-0.021981,-0.116683,-0.992490,-0.979640,-0.963321,-0.992563,-0.977304,-0.958142,-0.938850,...,-0.509523,-0.850065,-0.018043,0.092304,0.074220,-0.714534,-0.671943,-0.018351,-0.185733,SITTING
1,0.277440,-0.028086,-0.118412,-0.996620,-0.927676,-0.972294,-0.997346,-0.931405,-0.971788,-0.939837,...,-0.210792,-0.613367,-0.022456,-0.155414,0.247498,-0.112257,-0.826816,0.184489,-0.068699,STANDING
2,0.305833,-0.041023,-0.087303,0.006880,0.182800,-0.237984,0.005642,0.028616,-0.236474,0.016311,...,0.579587,0.394388,-0.362616,0.171069,0.576349,-0.688314,-0.743234,0.272186,0.053101,WALKING
3,0.276053,-0.016487,-0.108381,-0.995379,-0.983978,-0.975854,-0.995877,-0.985280,-0.974907,-0.941425,...,-0.566291,-0.841455,0.289548,0.079801,-0.020033,0.291898,-0.639435,-0.111998,-0.123298,SITTING
4,0.271998,0.016904,-0.078856,-0.973468,-0.702462,-0.869450,-0.979810,-0.711601,-0.856807,-0.920760,...,0.447577,0.214219,0.010111,0.114179,-0.830776,-0.325098,-0.840817,0.116237,-0.096615,STANDING


In [124]:
# data_test에서 x_test와 y_test를 분할합니다.

target = 'Activity'
x_test = data_test.drop(columns=target)  
y_test = data_test.loc[:,target]
# (1)y_test 라벨링 
activity_mapping = {
    'LAYING': 0,
    'STANDING': 1,
    'SITTING': 2,
    'WALKING': 3,
    'WALKING_UPSTAIRS': 4,
    'WALKING_DOWNSTAIRS': 5
}
y_test = y_test.map(activity_mapping) # target label 0~5 변환

# (2) x_test 학습용 데이터도 minmax 스케일링을 해줬기 때문에 test도 같은 환경(minmax ) 스케일링 
# scaler.fit(x_train) 학습이 
x_test = scaler.transform(x_test)#  평가용 데이터 역시 fit을 해선 안됀다 


# 각 모델의 성능 평가
models = {
    "KNN": best_knn,
    "SVM": best_svm,
    "Logistic Regression": best_log,
    "XGBoost": best_xgb,
    "lightGBM": best_lgbm
}

for model_name, model in models.items():
    # 예측 수행
    y_pred = model.predict(x_test)
    
    # 성능 평가
    print(f"============={model_name} classification_report =============")
    print(classification_report(y_test, y_pred))
    
    # 혼동 행렬 출력
    cm = confusion_matrix(y_test, y_pred)
    print("V=====Confusion Matrix=====V")
    print(cm)
    print("\n")

=============KNN classification_report =============
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       292
           1       0.94      0.97      0.95       287
           2       0.96      0.93      0.94       254
           3       1.00      1.00      1.00       228
           4       0.99      1.00      0.99       215
           5       1.00      0.99      0.99       195

    accuracy                           0.98      1471
   macro avg       0.98      0.98      0.98      1471
weighted avg       0.98      0.98      0.98      1471

V=====Confusion Matrix=====V
[[292   0   0   0   0   0]
 [  0 277  10   0   0   0]
 [  0  19 235   0   0   0]
 [  0   0   0 228   0   0]
 [  0   0   0   1 214   0]
 [  0   0   0   0   2 193]]


=============SVM classification_report =============
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       292
           1       0.98      0.98      0.98     

## ** 요약**


- (양정우님) [Accuracy Score]
1. KNN  : 98%
   - 최적의 파라미터: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
2. SVM  : 99%
   - 최적의 파라미터: {'C': 10 , 'kernel': 'rbf' , gammma : 0.1 }
3. Logistic Regress : 98%
   - 최적의 파라미터: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg' , max_iter = 100}
4. XGBoost  :99%
   - 최적의 파라미터: {'colsample_bytree': 0.8, 'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}

- (박슬기님)[Accuracy Score]
1. Logistic Regression => 98%
model_lr = LogisticRegression(
    solver='liblinear',
    penalty='l2',
    max_iter=500,
    l1_ratio=0.6842105263157894,
    C=11.288378916846883
)

2. KNN => 97%
model_knn = KNeighborsClassifier(
    weights='distance',
    n_neighbors=5,
    metric='manhattan'
)

3. Random Forest => 97%
model_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    max_features='sqrt',
    random_state=1)

4. Decision Tree => 97%
model_dt = DecisionTreeClassifier( 
    criterion='entropy',
    max_depth=30,
    min_samples_split=6,
    min_samples_leaf=3,
    random_state=1
)

- (김상아님)[Accuracy Score]
1. Logistic Regression 98%
model = LogisticRegression() # no params
2. DecisionTree 92%
max_depth=10
3. Random Forest 93%
max_depth = 5 

- (이찬울님) [Accuracy Score]
1. KNN => 96%
KNeighborsClassifier(n_neighbors=4)


2. DecisionTree => 94%
DecisionTreeClassifier(max_depth=8)

3.  lightGBM : 98% 
    param = {'n_estimators' : 260 ,'learning_rate':0.422 }

(윤성호님)[Accuracy Score]
  
1. SVM 98% model10 = SVC(kernel='linear', C=3)

2. Logistic:  97%

3. RandomForestClassifier(max_depth=5, n_estimators=100, random_state=1) : 94%

4. knn 93%

5. decison tree	model9 = DecisionTreeClassifier(max_depth=5)

(박소언님)[Accuracy Score]
1. KNN : 96%
n_neighbors = 5
metric = ‘manhattan’

2. Logistic Regression : 97%

3. SVM : 98%
- kernel = ‘linear’
- C  =  5

4. Random Forest : 97%
- max_depth 16

(진대규님) : 데이터 분할로 인해서 재 학습의 필요 
1. KNN = 92%  n_neighbors=5
2. Decision Tree = 86%  max_depth=5
3. Logistic Regression = 97%
4. SVM = 98%  kernel='linear', C=1
5. Random Forest = 91%  max_depth=5

## 결론 

DX_4반_11조 선정 모델 + 파라미터 (중간 점검) 

1. 로지스틱 회귀 모형  97~98%
    params =  {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg' , max_iter = 100}
2. SVM 서포트 벡터 머신 98~ 99%
    params =  {'C': 3 , 'kernel': 'linear'}
3. KNN  : 98%
    params =: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
4. lightGBM : 99% 
    param = {'n_estimators' : 260 ,'learning_rate':0.422 }


In [ ]:
# SVM , Logistic Regress , XGBoost는 모든 파라미터를 탐색하지 못했으므로 최적의 파라미터를 찾지 못함
# 오늘 미니프로젝트 이후 다시 모델생성할 예정 

## 5.모델 저장
* 각 알고리즘 별 최적의 성능 모델 저장
    * 단, 전체 변수를 이용해 생성한 모델만 저장합니다.(joblib.dump)
    * 튜닝 모델은, model.best_estimator_ 로 저장합니다.

    "KNN": best_knn,
    "SVM": best_svm,
    "Logistic Regression": best_log,
    "XGBoost": best_xgb,
    "lightGBM": best_lgbm

In [177]:
# 하이퍼파라미터들을 사용해서 모델 생성 & 학습 

# 1. LogisticRegression
model_log = LogisticRegression(C=10, penalty = 'l2', solver= 'newton-cg' , max_iter = 100)
model_log.fit(x_train, y_train)
# 2. Support Vector Machine (서포트 벡터 머신)
model_svm = SVC(C=3, kernel='linear')
model_svm.fit(x_train, y_train)
# 3. K-Nearest Neighbors (KNN)
model_knn = KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='distance')
model_knn.fit(x_train, y_train)
# 4. LightGBM (Light Gradient Boosting Machine)
model_lgbm = LGBMClassifier(n_estimators=260, learning_rate=0.422)
model_lgbm.fit(x_train, y_train)

LGBMClassifier(learning_rate=0.422, n_estimators=260)

In [178]:
joblib.dump(best_svm, 'best_svm_model.pkl')
joblib.dump(best_log, 'best_log_model.pkl')
joblib.dump(best_knn, 'best_knn_model.pkl')
joblib.dump(best_lgbm, 'best_lgbm_model.pkl')


['best_lgbm_model.pkl']

In [187]:
# 모델 불러오기 
model = joblib.load('best_lgbm_model.pkl')
y_pred = model.predict(x_test)

print('============평가 결과==================')
print(confusion_matrix(y_test, y_pred))
print('Classification Report  \n' , classification_report(y_test, y_pred ))

============평가 결과==================
[[291   0   0   0   0   1]
 [  0 281   6   0   0   0]
 [  0   5 249   0   0   0]
 [  0   0   0 227   1   0]
 [  0   0   0   0 215   0]
 [  0   0   0   1   0 194]]
Classification Report  
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       292
           1       0.98      0.98      0.98       287
           2       0.98      0.98      0.98       254
           3       1.00      1.00      1.00       228
           4       1.00      1.00      1.00       215
           5       0.99      0.99      0.99       195

    accuracy                           0.99      1471
   macro avg       0.99      0.99      0.99      1471
weighted avg       0.99      0.99      0.99      1471

